In [1]:
import networkx as nx
import numpy as np


In [14]:
# Create
edges = [(1,2), (2,4), (2,3), (3,2), (3,1), (4,1), (6,1), (6,3)]
nodes = [1,2,3,4,5,6]
G = nx.DiGraph(edges)
G.add_node(5)
print(G)

H = G.to_undirected()
print(H)

DiGraph with 6 nodes and 8 edges
Graph with 6 nodes and 7 edges


In [49]:
# Adjecentcy matrix 
adjaceny = nx.adjacency_matrix(G)

# Edge list 
edge_list = np.array([list(i) for i in list(G.edges)]).T


In [52]:
# Compute clustering coefficients 
nx.clustering(G)
nx.clustering(H)

dict_values([0.5, 0.6666666666666666, 1.0, 0.6666666666666666, 1.0, 0])

If you switch the labels of nodes 5 and 6 in Image 2.20a, how does that move change the adjacency matrix? And the link list?

Adjacency matrix 
swap row 4 and 5 and column 4 and 5.

Edge list 
Then we just change all instances of 6 to five and 5 to 6...? 

Node 5 is not in the adjacency list 

There are respectivly 5 and 0 paths. 


In [54]:
# Exercise 2.5 
edges = [(1,7), (2,9), (3,7), (3,8), (3,9), (4,9), (4,10), (5,9), (5,11), (6,11)]
G = nx.Graph(edges)
print(G)
# Max number of edges : size of first cluster times size of second. 
# (N_1^2 + N_2^2 + 2*N_1*N_2  - N_1 - N_2 ) / 2
# Difference : (N_1^2 + N_2^2 - N_1 - N_2 ) / 2  

# Because n_2 is large actual number of links divided by n_2 is large and goes towards 0.



Graph with 11 nodes and 10 edges


In [63]:
print(len(df_papers))

# Testing things 
for index, paper in df_papers.iterrows():
    print(ast.literal_eval(paper["field"]))
    break
    # ast.literal_eval(paper["field"])[0]["category"]
    # break

# for index, paper in df_papers.iterrows():
#     print(len(ast.literal_eval(paper["doi"])))

77349
[{'category': 'Physics', 'source': 's2-fos-model'}]


In [10]:
import pandas as pd
import ast
import networkx as nx
import numpy as np
from tqdm import tqdm


In [ ]:
# Function that takes in list of authorIds and outputs the corresponding top fields in the same order 
def author_field_v1(ids, df_author):
    """
    Compute the top field of the given authors 

    Args:
        ids (list): containing the ids of the authors in question
        df_author (pandas df): data frame of type author that contains the given ids 
    
    return: 
        fields (list): list of top fields of the given authors 
        n_CSS_authors (int): number of social science authors 
    """
    # Get boolean array that indicates where the authors are 
    mask = df_author["id"].isin(ids)
    
    # Get the fields 
    fields = list(df_author["field"][mask])
    
    # No. social science authors 
    n_CSS_authors = sum(mask)

    return fields, n_CSS_authors

# Not used because the len operator is good enough

In [15]:
# Function that takes in list of authorIds and outputs the corresponding top fields in the same order 
def author_field(ids, df_author):
    """
    Compute the top field of the given authors 

    Args:
        ids (list): containing the ids of the authors in question
        df_author (pandas df): data frame of type author that contains the given ids 
    
    return: 
        fields (list): list of top fields of the given authors 

    """
    # Get boolean array that indicates where the authors are 
    mask = df_author["id"].isin(ids)
    
    # Get the fields 
    fields = list(df_author["field"][mask]) 
    
    return fields 


In [ ]:
def argument_paper_dataframe_v1(df_papers, df_author):
    """
    Takes in a paper data frame, arguments it with 2 new columns and puts the fields of the author in the first column and the number of social scientists in the second.

    Args:
        df_papers (pandas dataframe): paper dataframe (like before)
        df_author (pandas dataframe): author dataframe (like before)

    Returns:
        df_papers (pandas dataframe): the dataframe from before, argumentet with two new columns
    """
    df_papers["author_field"] = None 
    df_papers["n_CSS_authors"] = None

    for index, row in tqdm(df_papers.iterrows()):
        # Currently authors are stored as a string representation of the list so we make it a list again 
        authors = ast.literal_eval(row["authors"]) 
        # Now we need to turn the authors into a list of integers, because the df_paper dataframe stores them as such 
        authors = [eval(id) for id in authors if id is not None]
        #authors = [eval(id) for id in authors]
        # Find the fields of the authors 
        authors_fields, n_CSS_authors = author_field(authors, df_author=df_author)
        df_papers["author_field"][index] = authors_fields
        df_papers["n_CSS_authors"][index] = authors_fields
        # print(authors_fields) # debugging 
    # print(df_papers.head()) # debugging 
        
    return df_papers 

# Not used because the len operator is good enough 

In [16]:
def argument_paper_dataframe(df_papers, df_author):
    """
    Takes in a paper data frame, arguments it with a new column and puts the fields of the author in that column.

    Args:
        df_papers (pandas dataframe): paper dataframe (like before)
        df_author (pandas dataframe): author dataframe (like before)

    Returns:
        df_papers (pandas dataframe): the dataframe from before, argumentet with the new column
    """
    df_papers["author_field"] = None 

    for index, row in tqdm(df_papers.iterrows()):
        # Currently authors are stored as a string representation of the list so we make it a list again 
        authors = ast.literal_eval(row["authors"]) 
        # Now we need to turn the authors into a list of integers, because the df_paper dataframe stores them as such 
        authors = [eval(id) for id in authors if id is not None]
        #authors = [eval(id) for id in authors]
        # Find the fields of the authors 
        authors_fields = author_field(authors, df_author=df_author)
        df_papers["author_field"][index] = authors_fields
        # print(authors_fields) # debugging 
    # print(df_papers.head()) # debugging 
        
    return df_papers 


In [21]:
# Clean the paper datafram to a social science dataframe
def generate_CCS_papers_1(df_papers, social_science_fields, quantitative_fields, verbose=False):
    
    rows_to_drop = []
    
    # Drop rows that are before 2008
    for index, row in df_papers.iterrows():
        if row["year"] < 2008: 
            rows_to_drop.append(index)
    if verbose: print(f"{len(rows_to_drop)} papers removed because they were to old. ({len(rows_to_drop)/len(df_papers)*100:.0f}%)")
    df_papers.drop(index=rows_to_drop, inplace=True) 
    rows_to_drop = []
    
        
    # Drop rows that do not contain a DOI
    for index, row in df_papers.iterrows():
        if row["doi"] == None: 
            rows_to_drop.append(index)
        else: 
            try:
                ast.literal_eval(row["doi"])[0]["DOI"]
            except: 
                rows_to_drop.append(index)
    if verbose: print(f"{len(rows_to_drop)} papers removed because they did not have a DOI. ({len(rows_to_drop)/len(df_papers)*100:.0f}%)")
    df_papers.drop(index=rows_to_drop, inplace=True) 
    rows_to_drop = []
    
    # Drop paper if it includes biology
    for index, row in df_papers.iterrows():
        # Go through the entire list of fields for each paper 
        for field in ast.literal_eval(row["field"]):
            if field["category"] == "Biology":
                rows_to_drop.append(index)
    if verbose: print(f"{len(rows_to_drop)} papers removed because biology was in the field. ({len(rows_to_drop)/len(df_papers)*100:.0f}%)")
    df_papers.drop(index=rows_to_drop, inplace=True) 
    rows_to_drop = []
    
    # Drop paper if the fields are not not included in social science fields
    for index, row in df_papers.iterrows():
        is_in_SCF = False 
        # Go through the entire list of fields for each paper 
        for field in ast.literal_eval(row["field"]):
            if field["category"] in social_science_fields:
                is_in_SCF = True
        
        # if the paper was not in social science fields drop it 
        if not is_in_SCF: 
            rows_to_drop.append(index)
    if verbose: print(f"{len(rows_to_drop)} papers removed because thier fields not in Social Science Fields. ({len(rows_to_drop)/len(df_papers)*100:.0f}%)")
    df_papers.drop(index=rows_to_drop, inplace=True) 
    rows_to_drop = []
    
    return df_papers


# It is time consuming to add a row in the dataframe, hence it helps that the dataframe is 100 times smaller 
def generate_CCS_papers_2(df_papers, social_science_fields, quantitative_fields, verbose=False):
    
    rows_to_drop = []
    
    # Drop the papers with more than 9 Computational Social Science authors? TODO what does she mean! 
    for index, row in df_papers.iterrows():
        if len(row["author_field"]) > 9: # Count authors, should I check if they are in the author data frame? That's gonna take a while
            rows_to_drop.append(index)
    if verbose: print(f"{len(rows_to_drop)} papers removed because there are more than 9 CSS authors. ({len(rows_to_drop)/len(df_papers)*100:.0f}%)")
    df_papers.drop(index=rows_to_drop, inplace=True) 
    rows_to_drop = []
    
    # Drop paper if the fields are not not included in quantitative data and authors aren't either
    for index, row in df_papers.iterrows():
        is_in_SCF = False 
        # Go through the entire list of fields for each paper 
        for field in ast.literal_eval(row["field"]):
            if field["category"] in quantitative_fields:
                is_in_SCF = True
        
        # Check if the authors are in the quantitative_fields
        for author_field in row["author_field"]: 
            if author_field in quantitative_fields: 
                is_in_SCF = True
       
        # if the paper was not in social science fields drop it 
        if not is_in_SCF: 
            rows_to_drop.append(index)
    if verbose: print(f"{len(rows_to_drop)} papers removed because thier fields and authors are not in Quantitative Fields. ({len(rows_to_drop)/len(df_papers)*100:.0f}%)")
    df_papers.drop(index=rows_to_drop, inplace=True) 
    rows_to_drop = []
    
    return df_papers
    

In [18]:
# Heuristic 
social_science_fields = {"Political Science", "Sociology", "Economics"}
quantitative_fields = {"Mathematics", "Physics", "Computer Science"}

# Load dataframes
df_author = pd.read_csv("df_author.csv")
df_papers = pd.read_csv("df_paper.csv")

# Drop papers, but not based on the authors
n_papers_before = len(df_papers)

print(f"Initially we have {n_papers_before} papers.")
df_papers = generate_CCS_papers_1(df_papers, social_science_fields, quantitative_fields, verbose=True)

# Papers removed so far 
print(f"There are {len(df_papers)} papers left of {n_papers_before} which is {100*len(df_papers)/n_papers_before:.2f}%") 

Initially we have 1078817 papers.
302900 papers removed because they were to old. (28%)
148196 papers removed because they did not have a DOI. (19%)
122867 papers removed because biology was in the field. (20%)
466782 papers removed because thier fields not in Social Science Fields. (92%)
There are 38072 papers left of 1078817 which is 3.53%


In [19]:
# Now the time consuming part 
# Argment paper dataframe 
df_papers = argument_paper_dataframe(df_papers=df_papers, df_author=df_author)

0it [00:00, ?it/s]C:\Users\david\AppData\Local\Temp/ipykernel_11632/3790806285.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_papers["author_field"][index] = authors_fields
38072it [00:41, 913.57it/s] 


In [22]:
# Drop papers based on authors 
df_papers = generate_CCS_papers_2(df_papers, social_science_fields, quantitative_fields, verbose=True)

print(f"There are {len(df_papers)} papers left of {n_papers_before} which is {100*len(df_papers)/n_papers_before:.2f}%") 

165 papers removed because there are more than 9 CSS authors. (0%)
32257 papers removed because thier fields and authors are not in Quantitative Fields. (85%)
There are 5650 papers left of 1078817 which is 0.52%


In [23]:
pd.DataFrame.to_csv(df_papers, "df_CSS_paper.csv")